In [1]:
import os # Arbeiten mit Pfaden
import numpy as np
from typing import Tuple
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.initializers import Constant
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import TensorBoard
from tf_utils.callbacks import ConfusionMatrix

In [2]:
MODELS_DIR = os.path.abspath('C:/Selbststudium/Udemy/Udemy_Tensorflow/models')
if not os.path.exists(MODELS_DIR):
    os.mkdir(MODELS_DIR)
MODEL_FILE_PATH = os.path.join(MODELS_DIR, 'mnist_model.h5')

LOGS_DIR = os.path.abspath('C:/Selbststudium/Udemy/Udemy_Tensorflow/logs')
if not os.path.exists(LOGS_DIR):
    os.mkdir(LOGS_DIR)
MODEL_LOG_DIR = os.path.join(LOGS_DIR, 'mnist_cm')

In [3]:
def get_dataset(num_features: int, num_classes: int) -> Tuple[Tuple[np.ndarray, np.ndarray], Tuple[np.ndarray, np.ndarray]]:
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    
    x_train = x_train.reshape(-1, num_features).astype(np.float32)
    x_test  = x_test.reshape(-1, num_features).astype(np.float32)
    
    y_train = to_categorical(y_train, num_classes=num_classes, dtype=np.float32)
    y_test  = to_categorical(y_test, num_classes=num_classes, dtype=np.float32)

    print(f'x train shape: {x_train.shape}')
    print(f'y train shape: {y_train.shape}')
    print(f'x test shape: {x_test.shape}')
    print(f'y test shape: {y_test.shape}')

    return (x_train, y_train), (x_test, y_test)

In [4]:
def build_model(num_features: int, num_classes: int) -> Sequential:
    init_w = TruncatedNormal(mean=0.0, stddev=0.01)
    init_b = Constant(value=0.0)

    model = Sequential()
    model.add(Dense(units=500, kernel_initializer=init_w, bias_initializer=init_b, input_shape=(num_features,))) 
        # Kernel waere die W-, bias die b-Matrix (Weights und Bias)
        # Müssen nicht angegeben werden, weil per Default praktische Zufallswerte gewählt werden
    model.add(Activation('relu'))
    model.add(Dense(units=250, kernel_initializer=init_w, bias_initializer=init_b)) # Hier kein input_shape mehr angeben! Nur beim ersten Layer 
    model.add(Activation('relu'))
    model.add(Dense(units=100, kernel_initializer=init_w, bias_initializer=init_b)) 
    model.add(Activation('relu'))
    model.add(Dense(units=50, kernel_initializer=init_w, bias_initializer=init_b)) 
    model.add(Activation('relu'))
    model.add(Dense(units=num_classes, kernel_initializer=init_w, bias_initializer=init_b))
    model.add(Activation('softmax'))
    model.summary()
    return model

In [5]:
def main() -> None:
    num_features = 784 # Bild hat 28*28 Pixel
    num_classes = 10 # 10 Ziffern möglich
    
    (x_train, y_train), (x_test, y_test) = get_dataset(num_features, num_classes)
    
    model = build_model(num_features, num_classes)

    opt = Adam(learning_rate=0.001)

    model.compile(
        loss='categorical_crossentropy', # wird bei Kategorie-Problemen mit mehr als 2 Klassen genommen
        optimizer=opt,
        metrics=['accuracy']
    )

    tb_callback = TensorBoard(
        log_dir=MODEL_LOG_DIR,
        histogram_freq=1,
        write_graph=True
    )

    classes_list = [class_idx for class_idx in range(num_classes)]

    cm_callback = ConfusionMatrix(
        model,
        x_test,
        y_test,
        classes_list=classes_list,
        log_dir=MODEL_LOG_DIR
    )

    model.fit(
        x=x_train,
        y=y_train,
        epochs=20,
        batch_size=256,
        verbose=1,
        validation_data=(x_test, y_test),
        callbacks=[tb_callback, cm_callback]
    )

    scores = model.evaluate(
        x=x_test, 
        y=y_test, 
        verbose=0
    )

In [6]:
if __name__=='__main__':
    main()

x train shape: (60000, 784)
y train shape: (60000, 10)
x test shape: (10000, 784)
y test shape: (10000, 10)
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 500)               392500    
                                                                 
 activation (Activation)     (None, 500)               0         
                                                                 
 dense_1 (Dense)             (None, 250)               125250    
                                                                 
 activation_1 (Activation)   (None, 250)               0         
                                                                 
 dense_2 (Dense)             (None, 100)               25100     
                                                                 
 activation_2 (Activation)   (None, 100)               0         
              